# Sentiment Analysis with RNN

In [1]:
# Import and data read

import numpy as np
import tensorflow as tf

with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

In [2]:
reviews[:200]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  '

In [3]:
# Removing punctuation

from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

all_text[:200]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s '

In [4]:
# Word examples

words[:5]

['bromwell', 'high', 'is', 'a', 'cartoon']

In [102]:
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


word_counter = Counter(words)
word_counter_cleaned = {k:v for k,v in word_counter.items() if k not in stopwords.words('english')}
words = [i[0] for i in Counter(word_counter_cleaned).most_common(1500)]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
# Word2int dictionary
vocab_to_int = {k:v for v, k in enumerate(set(words), start = 1)}

# Reviews to integers
reviews_ints = [[vocab_to_int[word] for word in review.split(' ') if word in vocab_to_int.keys()] for review in reviews]

# Labels to integers
labels = [1 if label == 'positive' else 0 for label in labels.split('\n')]

In [86]:
from collections import Counter

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))
print('Total reviews {}'.format(len(reviews_ints)))

Zero-length reviews: 1
Maximum review length: 603
Total reviews 25001


In [87]:
# Remove a review with the length of 0
zero_length_review = [i for i, j in enumerate(reviews_ints) if len(j) == 0]
for i in zero_length_review:
  reviews_ints.pop(i)
  labels.pop(i)

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))
print('Total reviews {}'.format(len(reviews_ints)))

Zero-length reviews: 0
Maximum review length: 603
Total reviews 25000


In [88]:
# Sequence padding (pre)

seq_len = 200

features = []
for review in reviews_ints:
  if len(review) > 200:
    features.append(review[:seq_len])
  else:
    while len(review) < 200:
      review.insert(0, 0)
    features.append(review)
    
print(features[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 447, 233, 817, 948, 905, 68, 372, 1115, 1340, 447, 854, 913, 1434, 179, 1218, 1144, 312, 1202, 1158, 1434, 343, 559, 353, 180, 905, 826, 447, 1141, 539, 713, 353, 447, 1051, 202, 118, 527, 114, 447, 112]


In [89]:
import random

# Train-test split
random.shuffle(reviews_ints)
split_frac = 0.8


train_x, val_x = np.array(features[:int(len(reviews_ints)*split_frac)]), np.array(features[int(len(reviews_ints)*split_frac):])
train_y, val_y = np.array(labels[:int(len(labels)*split_frac)]), np.array(labels[int(len(labels)*split_frac):])

val_x, test_x = np.array(val_x[:int(len(val_x)*0.5)]), np.array(val_x[int(len(val_x)*0.5):])
val_y, test_y = np.array(val_y[:int(len(val_y)*0.5)]), np.array(val_y[int(len(val_y)*0.5):])

print('\t\t\tFeature Shapes:')
print(f'Train set: \t\t{train_x.shape}',
       f'\nValidation set: \t{val_x.shape}',
       f'\nTest set: \t\t{test_x.shape}')

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Activation, TimeDistributed, Bidirectional, Embedding

# train_x = train_x.reshape(20000, 200, 1)
# train_y = train_y.reshape(20000, 1)

inputs = Input(shape = (train_x.shape[1:]))
gru_1 = GRU(200, return_sequences = True)(inputs)
gru_2 = GRU(200, return_sequences = True)(gru_1)
dense_1 = TimeDistributed(Dense(1, activation = 'sigmoid'))(gru_2)

model = Model(inputs = inputs, outputs = dense_1)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

NameError: name 'train_x' is not defined

In [97]:
history = model.fit(train_x, train_y, epochs = 200, batch_size = 128)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 0.8427 - accuracy: 0.5001
Epoch 2/200
1/1 [==============================] - 1s 751ms/step - loss: 0.9685 - accuracy: 0.4991
Epoch 3/200
1/1 [==============================] - 1s 746ms/step - loss: 0.8142 - accuracy: 0.5005
Epoch 4/200
1/1 [==============================] - 1s 740ms/step - loss: 0.8517 - accuracy: 0.5048
Epoch 5/200
1/1 [==============================] - 1s 759ms/step - loss: 0.8574 - accuracy: 0.5031
Epoch 6/200
1/1 [==============================] - 1s 775ms/step - loss: 0.8035 - accuracy: 0.5046
Epoch 7/200
1/1 [==============================] - 1s 795ms/step - loss: 0.7887 - accuracy: 0.5070
Epoch 8/200
1/1 [==============================] - 1s 736ms/step - loss: 0.8019 - accuracy: 0.5052
Epoch 9/200
1/1 [==============================] - 1s 759ms/step - loss: 0.8001 - accuracy: 0.5077
Epoch 10/200
1/1 [==============================] - 1s 759ms/step - loss: 0.7780 - accuracy: 0.5128
Epoch 11/200

1/1 [==============================] - 1s 743ms/step - loss: 0.6464 - accuracy: 0.6291
Epoch 84/200
1/1 [==============================] - 1s 746ms/step - loss: 0.6460 - accuracy: 0.6265
Epoch 85/200
1/1 [==============================] - 1s 725ms/step - loss: 0.6448 - accuracy: 0.6321
Epoch 86/200
1/1 [==============================] - 1s 742ms/step - loss: 0.6440 - accuracy: 0.6323
Epoch 87/200
1/1 [==============================] - 1s 734ms/step - loss: 0.6440 - accuracy: 0.6339
Epoch 88/200
1/1 [==============================] - 1s 732ms/step - loss: 0.6434 - accuracy: 0.6330
Epoch 89/200
1/1 [==============================] - 1s 730ms/step - loss: 0.6426 - accuracy: 0.6343
Epoch 90/200
1/1 [==============================] - 1s 723ms/step - loss: 0.6423 - accuracy: 0.6348
Epoch 91/200
1/1 [==============================] - 1s 751ms/step - loss: 0.6425 - accuracy: 0.6330
Epoch 92/200
1/1 [==============================] - 1s 745ms/step - loss: 0.6419 - accuracy: 0.6349
Epoch 93/200


In [95]:
train_x.shape

(1, 20000, 200)